In [1]:
import pandas as pd
import spacy
from spacy.lang.id import Indonesian
from spacy.lang.id.stop_words import STOP_WORDS
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import re

In [21]:
raw_data=pd.read_csv('detik_news_title.csv')

In [23]:
raw_data=raw_data.iloc[:,2:]

In [24]:
raw_data.head()

,title,category
0,Kemnaker Awasi TKA di Meikarta,finance
1,BNI Digitalkan BNI Java Jazz 2020,finance
2,"Terbang ke Australia, Edhy Prabowo Mau Genjot ...",finance
3,OJK Siapkan Stimulus Ekonomi Antisipasi Dampak...,finance
4,Saran Buat Anies-RK yang Mangkir Rapat Banjir ...,finance


# Preprocessing

In [5]:
import gdown
kamus_alay_url='https://drive.google.com/uc?id=14M5i-Dd_oFCCJf4avgTEQeZLO5PCKCzy'
kamus_alay_path='kamus_alay.csv'
kamus_singkatan_url='https://drive.google.com/uc?id=1f1aOazCYJXCnhg7Brf5GelE7yE8-QflK'
kamus_singkatan_path='kamus_singkatan.csv'
gdown.download(kamus_alay_url, kamus_alay_path, quiet=False)
gdown.download(kamus_singkatan_url, kamus_singkatan_path, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=14M5i-Dd_oFCCJf4avgTEQeZLO5PCKCzy
To: c:\Users\vince\Documents\Dokumen Vincent\Semester 6\Kapita Selekta Sistem Cerdas\kamus_alay.csv
100%|██████████| 3.09M/3.09M [00:02<00:00, 1.24MB/s]
Downloading...
From: https://drive.google.com/uc?id=1f1aOazCYJXCnhg7Brf5GelE7yE8-QflK
To: c:\Users\vince\Documents\Dokumen Vincent\Semester 6\Kapita Selekta Sistem Cerdas\kamus_singkatan.csv
100%|██████████| 18.9k/18.9k [00:00<00:00, 3.53MB/s]


'kamus_singkatan.csv'

In [6]:
kamus_alay=pd.read_csv(kamus_alay_path)
kamus_alay=kamus_alay.rename(columns = {'slang' : 'kamus_slang' , 'formal' : 'kamus_perbaikan'})
# Rekonstruksi data sebagai 'dict'
slang_mapping = dict(zip(kamus_alay['kamus_slang'], kamus_alay['kamus_perbaikan']))
kamus_singkatan = pd.read_csv(kamus_singkatan_path, header=None, names=['sebelum_perbaikan', 'setelah_perbaikan'],delimiter=';')
singkatan_mapping=dict(zip(kamus_singkatan['sebelum_perbaikan'],kamus_singkatan['setelah_perbaikan']))

In [8]:
# Menggunakan kamus kata gaul Salsabila
from Sastrawi.StopWordRemover.StopWordRemoverFactory import  StopWordRemoverFactory
import emoji
stopword_factory = StopWordRemoverFactory()
stopwords = stopword_factory.get_stop_words()
# List of words with negation meaning
data = emoji.EMOJI_DATA

# Remove negation words from stopwords
stopwords = set(stopwords)
nlp = Indonesian()
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [9]:
def preprocessing_tweet(tweet) :
    tweet=tweet.lower()
    tweet=re.sub(r'@[A-Za-z0-9]+','',tweet) # Remove Mentions
    tweet=re.sub(r'#[A-Za-z0-9]+','',tweet) # Remove Hashtags
    tweet=re.sub(r'RT[\s]+','',tweet) # Remove Retweets
    tweet=re.sub(r'https?:\/\/\S+','',tweet) # Remove Hyperlinks
    tweet=re.sub(r'www?:\/\/\S+','',tweet) # Remove Hyperlinks
    tweet=re.sub(r'\n','',tweet) # Remove Newlines
    tweet=re.sub(r'\r','',tweet) # Remove Newlines
    tweet=re.sub(r'\t','',tweet) # Remove Newlines
    tweet=re.sub(r'\s+',' ',tweet) # Remove Extra Spaces
    tweet=re.sub(r'\d+','',tweet) # Remove Numbers
    tweet=re.sub(r'[^a-zA-Z\s]','',tweet) # Remove Punctuations
    tweet=re.sub(r'\s+[a-zA-Z]\s+',' ',tweet) # Remove Single Characters
    tweet=re.sub(r'\s+[a-zA-Z]$','',tweet) # Remove Single Characters
    tweet=re.sub(r'^[a-zA-Z]\s+','',tweet) # Remove Single Characters
    tweet=re.sub(r'\s+',' ',tweet) # Remove Extra Spaces
    tweet=emoji.demojize(tweet) # Convert Emojis to Text
    tweet_tokens = []
    tokens = tweet.split()
    for ele in tokens:
      ele_kamus = kamus_singkatan.get(ele, ele)
      ele_slang = slang_mapping.get(ele_kamus, ele_kamus)
      tweet_tokens.append(ele_slang)

    tweet = ' '.join(tweet_tokens)
    tweet = nlp(tweet)
    tweet = [token.text for token in tweet if token.text not in stopwords]
    tweet = ' '.join(tweet)
    tweet = stemmer.stem(tweet)
    return tweet

In [25]:
raw_data['cleaned_title']=raw_data['title'].apply(preprocessing_tweet)

In [26]:
raw_data.to_csv("cleaned_dataset.csv",index=False)